# Multi-Class Classification

# Путешествие по Спрингфилду.


Сегодня вам предстоить помочь телекомпании FOX  в обработке их контента. Как вы знаете сериал Симсоны идет на телеэкранах более 25 лет и за это время скопилось очень много видео материала. Персоонажи менялись вместе с изменяющимися графическими технологиями   и Гомер 2018 не очень похож на Гомера 1989. Нашей задачей будет научиться классифицировать персонажей проживающих в Спрингфилде. Думаю, что нет смысла представлять каждого из них в отдельности.



 ![alt text](https://vignette.wikia.nocookie.net/simpsons/images/5/5a/Spider_fat_piglet.png/revision/latest/scale-to-width-down/640?cb=20111118140828)

### Импортирование нужных библиотек

In [ ]:
import os
import random
from pathlib import Path
from PIL import Image
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Загрузка файлов

Создадим функцию для загрузки файлов, который будет разделять файлы `train` на `train` и `val` и возвращать файлы для тренировки, валидации, теста и классы (42 штуки). Путем проб и ошибок становится понятно что `val_split` в 0.1 наиболее оптимален.

In [ ]:
def load_files(train_dir, test_dir, val_size=0.1):

    classes = sorted(os.listdir(train_dir))
    test_files = sorted(Path(test_dir).rglob('*.jpg'))
    train_val_files=sorted(Path(train_dir).rglob('*.jpg'))
            
    train_val_labels=[classes.index(path.parent.name) for path in train_val_files]
    train_files, val_files = train_test_split(train_val_files, test_size=val_size,
                                stratify=train_val_labels, random_state=42)
    return train_files, val_files, test_files, classes

#### Загрузим файлы

In [ ]:
train_dir = '../input/journey-springfield/train/simpsons_dataset/'
test_dir = '../input/journey-springfield/testset/testset/'

train_files, val_files, test_files, classes = load_files(train_dir, test_dir)

train_labels = [classes.index(path.parent.name) for path in train_files]
val_labels = [classes.index(path.parent.name) for path in val_files]

#### Создание датасета:

При создании датасета учитываем что при `mode=='test'` метки возвращать не нужно

In [ ]:
class DataSet(Dataset):

    def __init__(self, files, labels=None, transform=None, mode=None):
        self.files = files
        self.labels = labels
        self.transform = transform
        self.mode=mode

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx])
        image = self.transform(image)
        if self.mode == 'test':
            return image
        else:
            return image, self.labels[idx]

#### Определение трансформов для каждого сета:

При работе с различными вариантами трасформов, оказалось наиболее оптимальным решением использование только `RandomHorizontalFlip` (ну и ряда довольно дефолтных `Resize`, `Notmalize` и т.д.). Если быстро пробежаться по картинкам можно заметить, что почти во всех персонажи расположены под прямым углом и по центру. И дествительно использование `RandomResizedCrop`, `RandomAffine`, `ColorJitter` и т.д. лучших значении на тестовой выборке не дало при этом сходимость наблюдалась на более поздних эпохах. В силу нецелесообразности вышеупомянутых трансформов был использован только `RandomHorizontalFlip`. 

In [ ]:
input_size=(224, 224)

train_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

val_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

test_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

In [ ]:
train_set = DataSet(train_files, train_labels, train_transforms, 'train')
val_set = DataSet(val_files, val_labels, val_transforms, 'val')
test_set = DataSet(test_files, None, test_transforms, 'test')

print('Num samples in:\nTrain: {}\nVal: {}\nTest: {}'.format(len(train_set), len(val_set), len(test_set)))

Попробовав различные `batch_size` пришел к выводу что `batch_size=32` является наиболее подходящим.(Для наилучшей воспроизводимости ставим `shuffle=False`).

In [ ]:
train_gen = DataLoader(train_set, batch_size=32, shuffle=False, num_workers=4)
val_gen = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=4)
test_gen = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)

print('Num batches in:\nTrain: {}\nVal: {}\nTest: {}'.format(len(train_gen), len(val_gen), len(test_gen)))

### Определение модели

Мы будем использовать предобученный ResNet152. Замораживать слои мы не будем. И изменим только классификатор (последний FC слои) на такой же послносвязный только с нужным количеством классов на выходе. (Можно попробовать и ResNet18, который наиболее прост и потребляет не столько ресурсов как ResNet152. **Однако несмотря на то что ResNet152 обучается ~ 4 минуты/эпоху, а ResNet18 ~ 2 минуты/эпоху, ResNet152 дает больший скор за 5 эпох чем ResNet18 за 10. За одну эпоху ResNet152 дает ~ 0.81 на трейне и ~ 0.97 на валидации, что довольно впечетляюще.)**

**ОБУЧЕННУЮ МОДЕЛЬ МОЖНО НАЙТИ ПО ССЫЛКЕ:** [simpsons_resnet152.pth](https://www.kaggle.com/kanametov/add-to-simpsons#best_model_resnet152_v1.pth)

In [ ]:
model = models.resnet152(pretrained=True)

for param in model.parameters():
    param.requires_grad == True
    
model.fc = nn.Linear(2048, 42)
model=model.to(device)

### Определение весов

Поскольку некоторые классы имеют большее значение для данной задачи мы определим веса для каждого из классов. Все классы в которых имеется больше 200 сэмплов (картинок) в датасете будем причислять к **важным классам**. Для них веса будут больше. Вес класса входящего в список **важных** будет обратно зависеть от количества сэмплов в нем. Т.о. мы как бы выравниваем количество сэмплов в каждом из **важных классов** штрафуя больше те в котрых меньше картинок.

**ВЕСА МОЖНО НАЙТИ ПО ССЫЛКЕ:** [weights.csv](https://www.kaggle.com/kanametov/add-to-simpsons#final_weights(not_uniform).csv)

In [ ]:
def generate_weights(train_dir, classes):
    files = sorted(Path(train_dir).rglob('*.jpg'))
    important_classes=[]
    important_samples=[]
    num_samples_per_class={}
    for cls in classes:
        num_samples = len(sorted(Path(os.path.join(train_dir, cls)).rglob('*.jpg')))
        num_samples_per_class[cls] = num_samples
        if num_samples>200:
            important_classes.append(cls)
            important_samples.append(num_samples)

    weights=[]
    for cls in classes:
        if cls in important_classes:
            w = 0.05*(sum(important_samples)/(len(important_classes)*num_samples_per_class[cls]))
            weights.append(w)
        else:
            w = 0.5*(num_samples_per_class[cls]/sum(num_samples_per_class.values()))
            weights.append(w)
    norm_weights = [w/sum(weights) for w in weights]
    return norm_weights

In [ ]:
weights=generate_weights(train_dir, classes)
weights = torch.FloatTensor(weights)

### Определение функции потерь и оптимизатора

Попробовав различные оптимизаторы (`Adam`, `Adam + amsgrad`, `RMSProp`, `SGD`) пришел к выводу что `SGD` c циклическим `learning_rate` наиболее оптимален .(Параметры подобраны методом проб и ошибок. Для каждой модели стоит поискать наиболее подходящие)

In [ ]:
base_lr = 0.0018
max_lr = 0.0032
batch_size=32

criterion = nn.CrossEntropyLoss(weights).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov = True)
step_size = 2 * math.ceil(len(train_set)/batch_size)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr = base_lr, max_lr = max_lr,
                                              step_size_up=step_size, mode='exp_range', gamma=0.994,
                                              scale_mode='cycle', cycle_momentum=True, base_momentum=0.8, max_momentum=0.9, last_epoch=-1)


### Определение функции тренировки

In [ ]:
def train(train_gen, val_gen, epochs=5):
    val_acc = []
    val_loss = []
    train_acc = []
    train_loss = []

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch+1, epochs))
        print('-' * 10)
        model.train()  
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in tqdm(train_gen):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                loss.backward()
                optimizer.step()
                scheduler.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(train_gen.dataset)
        epoch_acc = running_corrects.double() / len(train_gen.dataset)
        train_acc.append(epoch_acc)
        train_loss.append(epoch_loss)
        
        print('Train loss: {:.4f} --- Train accuracy: {:.4f}'.format(epoch_loss, epoch_acc))

        model.eval()  
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in tqdm(val_gen):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(val_gen.dataset)
        epoch_acc = running_corrects.double() / len(val_gen.dataset)
        val_acc.append(epoch_acc)
        val_loss.append(epoch_loss)
        print('Validation loss: {:.4f} --- Validation accuracy: {:.4f}'.format(epoch_loss, epoch_acc))

    return {'loss': train_loss, 'acc': train_acc, 'val_loss': val_loss, 'val_acc': val_acc}

### Тренировка модели

**ОБУЧЕННУЮ МОДЕЛЬ МОЖНО НАЙТИ ПО ССЫЛКЕ:** [simpsons_resnet152.pth](https://www.kaggle.com/kanametov/add-to-simpsons#best_model_resnet152_v1.pth)

In [ ]:
history = train(train_gen, val_gen, epochs=5)

Можно также дообучить классификатор модели используя следующий код:
```python
for param in model.parameters():
    param.requires_grad == False
    
for param in model.fc.parameters():
    param.requires_grad == True
```
Однако это не дало лучшего скора, поэтому пользоваться этим подходом для данной модели мы не будем. (В ином случае стоит попробовать).

#### Сохранение модели:

In [ ]:
torch.save(model.state_dict(), 'best_model_resnet152_v1.pth')

### Результаты модели

In [ ]:
loss = history['loss']
acc = history['acc']
val_loss = history['val_loss']
val_acc = history['val_acc']

In [ ]:
plt.plot(range(1, len(loss)+1), loss, 'r-', label='train')
plt.plot(range(1, len(val_loss)+1), val_loss, 'b-', label='val')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(range(1, len(acc)+1), acc, 'r-', label='train')
plt.plot(range(1, len(val_acc)+1), val_acc, 'b-', label='val')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.grid()
plt.show()

### Проверка на тестовых данных

#### Функция предсказаний:

In [ ]:
def predict(test_gen):
    with torch.no_grad():
        logits = []
    
        for inputs in test_gen:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=1).numpy()
    return probs

In [ ]:
probs = predict(test_gen)
test_labels = np.argmax(probs, axis=1)
test_classes = [classes[i] for i in test_labels]

In [ ]:
my_submit = pd.DataFrame({'Id': sorted(os.listdir(test_dir)), 'Expected': test_classes})
my_submit.head()

In [ ]:
my_submit.to_csv('my_submission.csv', index=False)